In [ ]:
import socialsim as ss
import json
import pandas as pd
import numpy as np
import json
from glob import glob
import itertools
from dateutil import parser
from datetime import datetime

from collections import OrderedDict, defaultdict, deque, Counter
import pickle as pkl
from scipy.stats.stats import pearsonr, spearmanr
from scipy import stats, signal
from sklearn.kernel_ridge import KernelRidge
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.ticker import MaxNLocator
import seaborn as sns
import simplejson
import gzip
from math import sqrt
from scipy.optimize import curve_fit

from joblib import parallel_backend
from joblib import Parallel, delayed
from tqdm import tqdm
import os
import subprocess

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
sns.set()

In [ ]:
# articles = defaultdict(dict)
# with gzip.open('/data/leidos/2021CP5/CPEC/Exogenous/NewsArticles/cp5-cpec.exogenous.news-articles.v1.json.gz', 'r') as f:
#     for l in f:
#         tmp = json.loads(l)
#         if 'extension' in tmp and 'earliest_reference_datetime' in tmp['extension'] and 'title' in tmp and 'text' in tmp:
#             date = pd.to_datetime(tmp['extension']['earliest_reference_datetime']).date()
#             if tmp['title'] not in articles[date]:
#                 articles[date][tmp['title']] = [1, tmp['text']]
#             else:
#                 articles[date][tmp['title']][0] += 1

In [ ]:
json_data = []
with open('/home/dmg/2021CP5/news_results.json', 'r') as f:
    for l in f:
        tmp = json.loads(l)
        try:
            if pd.to_datetime(tmp['date']) >= pd.to_datetime('2020-03-30') and pd.to_datetime(tmp['date']) <= pd.to_datetime('2020-08-31'):
                json_data.append(tmp)
        except:
            pass

In [ ]:
# INSERT TEMPLATE
# with open('', 'r') as f:
#     for l in f:
#         tmp = json.loads(l)
#         try:
#             if pd.to_datetime(tmp['date']) >= pd.to_datetime('2020-03-30') and pd.to_datetime(tmp['date']) <= pd.to_datetime('2020-08-31'):
#                 json_data.append(tmp)
#         except:
#             pass

In [ ]:
articles = defaultdict(lambda: defaultdict(dict))
for r in json_data:
#     if r['relevance_prediction'] and r['leidos_bert_prediction'] is not None and r['cleaned_article'] is not None and r['cleaned_article'] != '':
    if r['leidos_bert_prediction'] is not None and r['cleaned_article'] is not None and r['cleaned_article'] != '':
        for infoid in r['leidos_bert_prediction']:
            if r['cleaned_title'] not in articles[infoid][pd.to_datetime(r['date']).date()]:
                articles[infoid][pd.to_datetime(r['date']).date()][r['cleaned_title']] = [1, r['cleaned_article']]
            else:
                articles[infoid][pd.to_datetime(r['date']).date()][r['cleaned_title']][0] += 1

In [ ]:
tokens = defaultdict(dict)
for k, v in articles.items():
    for kk, vv in tqdm(v.items(), total=len(v)):
        tokens[k][kk] = word_tokenize(' '.join([' '.join([vvv[1]] * int(sqrt(vvv[0]))) for vvv in vv.values()]))

In [ ]:
# print('\n'.join([str(x) for x in articles[pd.to_datetime('2020-08-13').date()].keys()]))

In [ ]:
stop_words = set(stopwords.words('english') + pd.read_csv('newstop.csv').term.to_list()) 
words = set(nltk.corpus.words.words())

In [ ]:
ctss = defaultdict(dict)
for k, v in tokens.items():
    for kk, vv in tqdm(v.items(), total=len(v)):
        ctss[k][kk] = nltk.FreqDist([vvv.lower() for vvv in vv if len(vvv) > 3 and vvv.lower() not in stop_words and vvv.lower() in words])

In [ ]:
# for k1, v1 in ctss.items():
#     print(k1)
#     for i, (k, v) in enumerate(sorted(v1.items())):
#         print(i)
#         plt.figure(figsize=(18, 5))
#         plt.title(k.strftime('%m/%d/%Y'))
#         fig = v.plot(100).get_figure()
#         fig.tight_layout()
#         plt.show()
#     fig.savefig('entropy/{0:03d}.png'.format(i))
#     plt.close('all')

In [ ]:
etps = defaultdict(dict)
for k1, v1 in ctss.items():
    for k, v in sorted(v1.items()):
        tmp = v.most_common(100)
        s = sum([x[1] for x in tmp])
        ps = [x[1] / s for x in tmp]
        H = -sum([x * np.log(x) for x in ps])
        etps[k1][k] = -H

In [ ]:
def recifunc(x, s, c):
    return (1 / np.power(x, s)) / np.sum([1 / np.power(i, s) for i in range(1, 101)]) + c

In [ ]:
etps2 = defaultdict(dict)
for k1, v1 in ctss.items():
    for k, v in sorted(v1.items()):
        try:
            tmp = v.most_common(100)
            s = sum([x[1] for x in tmp])
            ps0 = [x[1] for x in tmp]
            ps = [x[1] / s for x in tmp]
            popt, pcov = curve_fit(recifunc, np.linspace(1, len(ps), len(ps)), ps)
            etps2[k1][k] = popt[0]
        except:
            print(k1, k, tmp)

In [ ]:
append = 'to_7_27'
with open('/data/leidos_extracted/2021CP5/twitter_time_series_{}.json'.format(append), 'r') as f:
    d = json.loads(f.read())
tdf = {k: pd.read_json(v, orient='columns').EventCount for k, v in d.items()}

In [ ]:
with open('/data/leidos_extracted/2021CP5/cp5_eval_nodes.txt', 'r') as f:
    nodes = f.read().split('\n')[:-2]
with open('/data/leidos_extracted/2021CP5/cp5_other_nodes.txt', 'r') as f:
    nodes += f.read().split('\n')[:-1]

In [ ]:
with open('/data/leidos_extracted/2021CP5/gdelt_time_series.json', 'r') as f:
    gdelt_ts = json.load(f)
gdelt = OrderedDict(sorted({k: pd.read_json(v, typ='series') for k, v in gdelt_ts.items()}.items(), key=lambda kv: kv[1].sum(), reverse=True))

In [ ]:
with open('/home/dmg/2021CP5/news_west_300_time_series_to_6_29.json', 'r') as f:
    west_ts = json.load(f)
west = OrderedDict(sorted({k: pd.read_json(v, typ='series') for k, v in west_ts.items()}.items(), key=lambda kv: kv[1].sum(), reverse=True))

In [ ]:
with open('/home/michael/dryrun/corrmap.json', 'r') as f:
    twitter_corr_data = json.load(f)['twitter']
twitter_corr_data = OrderedDict(
        sorted({
            k: OrderedDict(
                sorted(
                    v.items(), 
                    key=lambda kv:(kv[1], kv[0]), 
                    reverse=True
                )
            ) for k, v in twitter_corr_data.items()
        }.items())
    )

In [ ]:
# with open('/data/leidos_extracted/2021CP5/cp5_eval_nodes.txt', 'r') as f:
#     nodes = f.read().split('\n')[:-2]
idx = pd.date_range(pd.to_datetime('2020-03-30'), pd.to_datetime('2020-08-30'))

In [ ]:
for k, v in etps2.items():
    if len(v) == 0:
        print(k, v)

In [ ]:
etps2_df = {}
for k, v in etps2.items():
    tmpmean = np.mean(list(v.values()))
    tmp = pd.Series(v).reindex(idx, fill_value=tmpmean)
    tmp[tmp < 1e-3] = tmpmean
    etps2_df[k] =tmp

In [ ]:
for k in set(nodes) - set(etps2_df.keys()):
    etps2_df[k] = pd.Series().reindex(idx, fill_value=1)

In [ ]:
etps2_df_json = {k: v.to_json() for k, v in etps2_df.items()}

In [ ]:
with open('/data/leidos_extracted/2021CP5/zipf_time_series_to_8_31.json', 'w') as f:
    f.write(json.dumps(etps2_df_json))